In [ ]:
def showrst(df):
    print(df.to_markdown(tablefmt="grid"))

In [ ]:
import pandas as pd
import nannyml as nml
reference, analysis, analysis_gt = nml.datasets.load_synthetic_sample()
reference.head(3)

In [ ]:
analysis.head(3)

In [ ]:
analysis.head(3)
metadata = nml.extract_metadata(reference)
metadata.target_column_name = 'work_home_actual'

In [ ]:
cbpe = nml.CBPE(model_metadata=metadata, chunk_size=5000)
cbpe.fit(reference_data=reference)

In [ ]:
est_perf = cbpe.estimate(pd.concat([reference, analysis]))

In [ ]:
est_perf.data.head(3)

In [ ]:
fig = est_perf.plot(kind='performance')
fig.show()


In [ ]:
fig = est_perf.plot(kind='performance')
fig.write_image(file="../_static/performance_estimation_guide_synth.svg", engine="orca")

In [ ]:
analysis_gt.head(3)

In [ ]:
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
# merge gt to analysis
analysis_full = pd.merge(analysis, analysis_gt, on = 'identifier')
df_all = pd.concat([reference, analysis_full]).reset_index(drop=True)
target_col = 'work_home_actual'
pred_score_col = 'y_pred_proba'
actual_performance = []
for idx in est_perf.data.index:
    start_index, end_index = est_perf.data.loc[idx, 'start_index'], est_perf.data.loc[idx, 'end_index']
    sub = df_all.loc[start_index:end_index]
    actual_perf = roc_auc_score(sub[target_col], sub[pred_score_col])
    est_perf.data.loc[idx, 'actual_roc_auc'] = actual_perf
# plot
est_perf.data[['estimated_roc_auc', 'actual_roc_auc']].plot()
plt.xlabel('chunk')
plt.ylabel('ROC AUC')
plt.show()